In [7]:
import pyfirmata
import matplotlib.pyplot as plt
import time
from datetime import datetime
import pandas as pd

class SharpSensor():
    '''measuring sharp sensor through pyfirmata with arduino uno'''
    def __init__(self, analogue__input_pin = 5, digital_output_pin = 4,usb_port = '/dev/ttyUSB0',verbose = False, samples = 100, error_threshold = 0.05):
        '''start setting up pins and board'''
        ## initialize board
        self.board = pyfirmata.Arduino(usb_port)
        ## start board firmata
        it = pyfirmata.util.Iterator(self.board)
        it.start()
        ## set pins
        self.digital_output_pin = self.board.get_pin('d:{}:o'.format(digital_output_pin))
        self.analog_input_pin = self.board.get_pin('a:{}:i'.format(analogue__input_pin))
        ## set verbosity
        self.verbose = verbose
        ## samples
        self.samples = samples
        ## error threshold
        self.error_threshold = error_threshold
        ## set default write pin to high
        self.digital_output_pin.write(1)
        
    def read_data(self):
        self.digital_output_pin.write(1)
        time.sleep(1)
        raw_data = {'timeStamp':[],'voltage':[],'error':[],'i':[]}
        for i in range(self.samples):
            self.digital_output_pin.write(0)          #Turn on the dust sensor LED by setting digital pin LOW.
            start_time = datetime.now()
            time.sleep(0.00026)
            for _ in range(100):
                current_reading = self.analog_input_pin.read()
                timestamp = (datetime.now()-start_time).microseconds/1000
                if current_reading > self.error_threshold:
                    if self.verbose > 0:
                        print('output is {} V , time between reading is : {} ms'.format(current_reading*5,timestamp))
                    raw_data['timeStamp'].append(timestamp)
                    raw_data['voltage'].append(current_reading*5)
                    raw_data['error'].append(False)
                    raw_data['i'].append(i)
                    break
                else:
                    if self.verbose > 1 :
                        print('output is {} V , time between reading is : {} ms'.format(current_reading*5,timestamp))
                    raw_data['timeStamp'].append(timestamp)
                    raw_data['voltage'].append(current_reading*5)
                    raw_data['error'].append(True)
                    raw_data['i'].append(i)
                time.sleep(0.000005)
            self.digital_output_pin.write(1) 
            time.sleep(0.01)
        self.raw_data = pd.DataFrame(raw_data)
        self.non_error_raw_data = self.raw_data[self.raw_data['error']== 0]
        self.latest_reading = self.non_error_raw_data['voltage'].mean()
        
    def read_datas(self,n = 10):
        readingData = {'time_stamp':[],'reading(V)':[]}
        for _ in range (n):
            sensor.read_data()
            readingData['reading(V)'].append(sensor.latest_reading)
            readingData['time_stamp'].append(datetime.now())
        self.historicalData = pd.DataFrame(readingData).dropna()
        self.meanHistoricalData = self.historicalData['reading(V)'].mean()
        return self.historicalData
    def plot_latest(self):
        self.raw_data.plot(x='timeStamp',y='voltage',kind = 'scatter')
        plt.show()

In [8]:
sensor = SharpSensor()

In [6]:
sensor.read_datas(10)

,time_stamp,reading(V)
0,2019-12-13 01:22:32.362944,0.599205
1,2019-12-13 01:22:34.451161,0.593735
2,2019-12-13 01:22:36.539645,0.591645
3,2019-12-13 01:22:38.628874,0.595930
4,2019-12-13 01:22:40.717802,0.606100
5,2019-12-13 01:22:42.807129,0.597205
6,2019-12-13 01:22:44.897715,0.596415
7,2019-12-13 01:22:46.987459,0.585730
8,2019-12-13 01:22:49.077214,0.584890
9,2019-12-13 01:22:51.166993,0.589060


In [9]:
sensor.read_datas(10)

,time_stamp,reading(V)
0,2019-12-13 01:24:06.457725,0.489675
1,2019-12-13 01:24:08.545920,0.505060
2,2019-12-13 01:24:10.633776,0.488895
3,2019-12-13 01:24:12.722639,0.500235
4,2019-12-13 01:24:14.821687,0.509045
5,2019-12-13 01:24:16.920320,0.505020
6,2019-12-13 01:24:19.021942,0.502915
7,2019-12-13 01:24:21.123090,0.513190
8,2019-12-13 01:24:23.222557,0.491980
9,2019-12-13 01:24:25.321457,0.497950


In [10]:
sensor.read_datas(10)

,time_stamp,reading(V)
0,2019-12-13 01:27:58.969559,0.509155
1,2019-12-13 01:28:01.058410,0.592415
2,2019-12-13 01:28:03.147562,0.590510
3,2019-12-13 01:28:05.235044,0.582380
4,2019-12-13 01:28:07.321621,0.615290
5,2019-12-13 01:28:09.410308,0.596450
6,2019-12-13 01:28:11.497749,0.581700
7,2019-12-13 01:28:13.586184,0.555275
8,2019-12-13 01:28:15.674245,0.542795
9,2019-12-13 01:28:17.763117,0.553165


In [11]:
sensor.read_datas(10)

,time_stamp,reading(V)
0,2019-12-13 01:28:43.230295,0.608665
1,2019-12-13 01:28:45.319254,0.588205
2,2019-12-13 01:28:47.402810,0.574410
3,2019-12-13 01:28:49.507815,0.552255
4,2019-12-13 01:28:51.607486,0.566800
5,2019-12-13 01:28:53.711933,0.536090
6,2019-12-13 01:28:55.817264,0.540110
7,2019-12-13 01:28:57.903943,0.571575
8,2019-12-13 01:28:59.983538,0.595475
9,2019-12-13 01:29:02.087125,0.545060


In [6]:
import sys
ser = serial.Serial("/dev/ttyUSB1", baudrate=9600, stopbits=1, parity="N", timeout=2)
while True:
   line = ser.readline()
   if len(line) == 0:
      print("Time out! Exit.\n")
      sys.exit()
   print( line)

Time out! Exit.



SystemExit: 

/home/pi/gpio/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
import serial, time, struct

ser = serial.Serial("/dev/ttyUSB1", baudrate=9600, stopbits=1, parity="N", timeout=2)

ser.flushInput()

byte, lastbyte = "\x00", "\x00"

for _ in range(100):
    lastbyte = byte
    byte = ser.read(size=1)
    print(byte)
    
    # We got a valid packet header
    if lastbyte == "\xAA" and byte == "\xC0":
        sentence = ser.read(size=8) # Read 8 more bytes
        readings = struct.unpack('<hhxxcc',sentence) # Decode the packet - big endian, 2 shorts for pm2.5 and pm10, 2 reserved bytes, checksum, message tail
        
        pm_25 = readings[0]/10.0
        pm_10 = readings[1]/10.0

b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''


KeyboardInterrupt: 